In [8]:
import mediapipe as mp
import cv2
import csv
import pickle
import math
import numpy as np
import pandas as pd
import warnings
# from gtts import gTTS
import os
import time
from asgiref.sync import async_to_sync
from django.conf import settings
from asgiref.sync import async_to_sync
from channels.layers import get_channel_layer
from django.http import HttpResponse
import pyttsx3

# TODO: add new pose from admin

warnings.filterwarnings("ignore")

with open('../mediapipe/tree.pkl', 'rb') as f:
    model = pickle.load(f)

mp_drawing = mp.solutions.drawing_utils  # Drawing helpers
mp_pose = mp.solutions.pose  # Mediapipe Solutions

landmark_names = [
    'nose',
    'left_eye_inner', 'left_eye', 'left_eye_outer',
    'right_eye_inner', 'right_eye', 'right_eye_outer',
    'left_ear', 'right_ear',
    'mouth_left', 'mouth_right',
    'left_shoulder', 'right_shoulder',
    'left_elbow', 'right_elbow',
    'left_wrist', 'right_wrist',
    'left_pinky_1', 'right_pinky_1',
    'left_index_1', 'right_index_1',
    'left_thumb_2', 'right_thumb_2',
    'left_hip', 'right_hip',
    'left_knee', 'right_knee',
    'left_ankle', 'right_ankle',
    'left_heel', 'right_heel',
    'left_foot_index', 'right_foot_index'
]

joints = [
    'left_elbow',
    'right_elbow',
    'right_shoulder',
    'left_shoulder',
    'left_knee',
    'right_knee',
    'head',
    'left_hip',
    'right_hip',
    'left_ankle',
    'right_ankle'
]


# assign tree actual angles from csv file
def actual_refrence_angles(pose_name):

    tree_pose_angles = {
        x+"_angle": None for x in joints
    }

    keys = [x+"_angle" for x in joints]

    with open('data.csv', 'r') as file:
        reader = csv.reader(file)
        i = 0
        for row in reader:
            if row[0] == pose_name:
                for i in range(1, len(row)):
                    tree_pose_angles.update({keys[i-1]: float(row[i])})

        return tree_pose_angles


def return_angle(landmark1, landmark2, landmark3):
    x1, y1 = landmark1.x, landmark1.y
    x2, y2 = landmark2.x, landmark2.y
    x3, y3 = landmark3.x, landmark3.y

    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) -
                         math.atan2(y1 - y2, x1 - x2))

    if angle < 0:
        angle += 360

    # Return the calculated angle.
    return angle


def compute_joint_angles(pose):
    left_elbow_angle = return_angle(pose[landmark_names.index('left_wrist')],
                                    pose[landmark_names.index('left_elbow')],
                                    pose[landmark_names.index('left_shoulder')])
    right_elbow_angle = return_angle(pose[landmark_names.index('right_shoulder')],
                                     pose[landmark_names.index('right_elbow')],
                                     pose[landmark_names.index('right_wrist')])
    left_shoulder_angle = return_angle(pose[landmark_names.index('left_elbow')],
                                       pose[landmark_names.index(
                                           'left_shoulder')],
                                       pose[landmark_names.index('left_hip')])
    right_shoulder_angle = return_angle(pose[landmark_names.index('right_hip')],
                                        pose[landmark_names.index(
                                            'right_shoulder')],
                                        pose[landmark_names.index('right_elbow')])
    left_hip_angle = return_angle(pose[landmark_names.index('left_knee')],
                                  pose[landmark_names.index('left_hip')],
                                  pose[landmark_names.index('left_shoulder')])
    right_hip_angle = return_angle(pose[landmark_names.index('right_shoulder')],
                                   pose[landmark_names.index('right_hip')],
                                   pose[landmark_names.index('right_knee')])
    left_knee_angle = return_angle(pose[landmark_names.index('left_ankle')],
                                   pose[landmark_names.index('left_knee')],
                                   pose[landmark_names.index('left_hip')])
    right_knee_angle = return_angle(pose[landmark_names.index('right_hip')],
                                    pose[landmark_names.index('right_knee')],
                                    pose[landmark_names.index('right_ankle')])

    computed_angles = {'left_elbow_angle': left_elbow_angle,
                       'right_elbow_angle': right_elbow_angle,
                       'left_shoulder_angle': left_shoulder_angle,
                       'right_shoulder_angle': right_shoulder_angle,
                       'left_hip_angle': left_hip_angle,
                       'right_hip_angle': right_hip_angle,
                       'left_knee_angle': left_knee_angle,
                       'right_knee_angle': right_knee_angle}
    return computed_angles


def error_messages():
    arms_err_msgs = ['move left arm up', 'move left arm down',
                     'move right arm up', 'move right arm down',
                     'straighten your left arm', 'straighten your right arm']

    legs_err_msgs = ['move left leg up', 'move left leg down',
                     'move right leg up', 'move right leg down'
                     'straighten your left leg', 'straighten your right leg'
                     'bend your knees', 'bend your left knee', 'bend your right knee', 'move your legs apart',
                     'bring your legs closer']

    hips_error_message = ['lean to the left',
                          'lean to the right', 'stand straight']

    back_error_message = ['bend your back', 'straighten your back']

    head_error_message = ['put your head straight']


def get_pose_prediction(pose):
    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z,
                    landmark.visibility] for landmark in pose]).flatten())
    X = pd.DataFrame([pose_row])
    pose_detection_class = model.predict(X)[0]
    pose_detection_probability = model.predict_proba(X)[0]
    return pose_detection_class, pose_detection_probability


def generate_errors(pose_name, pose):
    # calculate the angles
    tree_pose_angles = actual_refrence_angles("tree")
    live_angles = compute_joint_angles(pose)
    max_diff = 0
    diff_joint = ""
    if pose_name == "tree":
        for angles in live_angles.keys():

            if tree_pose_angles[angles]:               
                
                diff = abs(live_angles[angles] - tree_pose_angles[angles])
                print(tree_pose_angles[angles],live_angles[angles],diff, angles)
                if diff > abs(max_diff):
                    max_diff = live_angles[angles] - tree_pose_angles[angles]
                    diff_joint = angles
    return max_diff, diff_joint


def get_coordinate(joint_name, landmarks):
    joint_dict = {'nose': 0, 'left_eye_inner': 1, 'left_eye': 2, 'left_eye_outer': 3, 'right_eye_inner': 4,
                  'right_eye': 5, 'right_eye_outer': 6, 'left_ear': 7, 'right_ear': 8, 'mouth_left': 9,
                  'mouth_right': 10, 'left_shoulder': 11, 'right_shoulder': 12, 'left_elbow': 13, 'right_elbow': 14,
                  'left_wrist': 15, 'right_wrist': 16, 'left_pinky': 17, 'right_pinky': 18, 'left_index': 19,
                  'right_index': 20, 'left_thumb': 21, 'right_thumb': 22, 'left_hip': 23, 'right_hip': 24,
                  'left_knee': 25, 'right_knee': 26, 'left_ankle': 27, 'right_ankle': 28, 'left_heel': 29,
                  'right_heel': 30, 'left_foot_index': 31, 'right_foot_index': 32}
    if joint_name in joint_dict:
        return landmarks[joint_dict[joint_name]][0], landmarks[joint_dict[joint_name]][1]
    else:
        return None


In [3]:
arms_err_msgs = ['move left arm up', 'move left arm down',
                 'move right arm up', 'move right arm down',
                 'straighten your left arm', 'straighten your right arm']

legs_err_msgs = ['move left leg up', 'move left leg down',
                 'move right leg up', 'move right leg down'
                 'straighten your left leg', 'straighten your right leg'
                 'bend your knees', 'bend your left knee', 'bend your right knee', 'move your legs apart',
                 'bring your legs closer']

hips_error_message = ['lean to the left',
                      'lean to the right', 'stand straight']

back_error_message = ['bend your back', 'straighten your back']

head_error_message = ['put your head straight']


In [16]:

cap = cv2.VideoCapture(0)
curr_time = 0
name=""
difference=0
error=""
start_time = time.time() 

# pose model
with mp_pose.Pose() as pose_tracker:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        result = pose_tracker.process(image)
        pose_landmarks = result.pose_landmarks

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        height, width, _ = image.shape
        landmarks = []
        if result.pose_landmarks:
            for landmark in result.pose_landmarks.landmark:

                # Append the landmark into the list.
                landmarks.append(
                    (int(landmark.x * width), int(landmark.y * height)))

       # print(landmarks)

        try:
            pose = pose_landmarks.landmark

            pose_name, score_arr = get_pose_prediction(pose)
            # print( pose_name)

            score = round(score_arr[np.argmax(score_arr)], 2)
            line_color = mp_drawing.DrawingSpec(
                color=(0, 0, 150), thickness=2, circle_radius=2)
            circle_color = mp_drawing.DrawingSpec(
                color=(0, 0, 200), thickness=2, circle_radius=2)

            # put redirect bhako link ko number instead of pose name for the website
            if pose_name == "tree" and score > 0.72:
                line_color = mp_drawing.DrawingSpec(
                    color=(0, 150, 0), thickness=2, circle_radius=2)
                circle_color = mp_drawing.DrawingSpec(
                    color=(0, 200, 0), thickness=2, circle_radius=2)

                if curr_time > 0:
                    print("Maintain pose")
                    curr_time = 0

            elif pose_name == "tree" and score < 0.72:
                if curr_time > 0:
                    difference, name = generate_errors(pose_name, pose)
                    # print(difference, name)
                  #  print("Improve the pose")
                    curr_time = 0

            else:
                if curr_time >= 50:
                  #  print("Improve Pose")
                    curr_time = 0

                   # print("kuch hua nai: Improve Pose")

            coords = (100, 100)

            # Get status box
            cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

            # Display Class
            cv2.putText(image, 'CLASS', (95, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, pose_name.split(' ')[
                        0], (90, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display Probability
            cv2.putText(image, 'PROB', (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(score), (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            if (name == 'left_shoulder_angle'):
                if (difference > 0):
                    error = arms_err_msgs[0]
                else:
                    error = arms_err_msgs[1]
            elif (name == 'right_shoulder_angle'):
                if (difference > 0):
                    error = arms_err_msgs[2]
                else:
                    error = arms_err_msgs[3]
            elif (name == 'left_elbow_angle'):
                error = "straighten your left arm"
            elif (name == 'right_elbow_angle'):
                error = "straighten your right arm"
            elif (name == 'left_hip_angle'):

                error = hips_error_message[0]
            elif (name == 'right_hip_angle'):
                error = hips_error_message[1]
            elif (name == 'left_knee_angle'):

                error = legs_err_msgs[8]
            elif (name == 'right_knee_angle'):
                error = legs_err_msgs[9]
            else:
                pass
               # print('detect bhayena', name, difference)

            cv2.putText(image, error, (get_coordinate("_".join(name.split("_")[:2]), landmarks)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # display error

            if pose_landmarks is not None:
                mp_drawing.draw_landmarks(
                    image,
                    pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    line_color,
                    circle_color
                )

        except Exception as e:
            print(e)
            if curr_time > 100:
                print("No pose detected")
                curr_time = 0

        curr_time += 1
        current_time = time.time()
        elapsed_time = current_time - start_time
                #yo jati chaincha rakhne
        if elapsed_time >= 20:
            print("sakyo")
            break
            
        cv2.imshow('Pose Detection', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


149.75225852305002 254.06424298798186 104.31198446493184 left_elbow_angle
208.05137547584283 250.48694017290265 42.43556469705982 right_elbow_angle
181.38598673331313 131.99393216591324 49.392054567399896 left_shoulder_angle
178.95560961007578 145.80767514436738 33.1479344657084 right_shoulder_angle
328.21624646738456 182.26837108653805 145.9478753808465 left_hip_angle
178.35890228780318 179.44502597625834 1.0861236884551602 left_knee_angle
132.33790416658093 182.2947400980821 49.95683593150116 right_knee_angle
149.75225852305002 247.84678664933693 98.09452812628692 left_elbow_angle
208.05137547584283 245.95518247234028 37.90380699649745 right_elbow_angle
181.38598673331313 144.0535514447843 37.33243528852884 left_shoulder_angle
178.95560961007578 152.1125145212414 26.84309508883439 right_shoulder_angle
328.21624646738456 181.96804239686276 146.2482040705218 left_hip_angle
178.35890228780318 179.61375293379507 1.254850645991894 left_knee_angle
132.33790416658093 182.97879064550057 50.6